# Capstone project: Pedicting HDB Prices

#### Objectives/ Problem Statement
To predict the price of new HDB launch before the launch <br>
_**Input a set of features and output a price **_

#### why this problem?
A practical problem that most Singaporeans (including myself) face 

## Part3_ Modelling_Round2 
- Using MRT and Resale Data
- y variable is the variance (take away the mean) 
- X data from 2005 onwards
- train set: 2005 to 2014
- test set : 2014 to 2018


**[Iteration 1: ](#iter1)** [SLR](#1slr)  ||  [MLR by category](#1mlrcat) <br> 
X = resale data <br>
y = resale['var_resale_price'] <br>

**[Iteration 2: ](#iter2)** [SLR](#2slr)  ||  [MLR by category](#2mlrcat) <br>
X = resale data <br>
y = resale['var_adj_price'] <br>


**[Iteration 3](#iter3)** <br>
X = resale data without time data<br>
y = resale['var_resale_price'] <br>
**[Iter3a - no yr:       ](#iter3a)**[MLR all](#3amlrall) || [ridge,lasso,enet](#3aregularization) <br>
**[Iter3b - no yr&month: ](#iter3b)**:[MLR all](#3bmlrall) || [ridge,lasso,enet](#3bregularization)

**[Iteration 4](#iter4)** <br>
X = resale data without time data<br>
y = resale['var_adj_price'] <br>
**[Iter4a - no yr:       ](#iter4a)**[MLR all](#4amlrall) || [ridge,lasso,enet](#4aregularization) <br>
**[Iter4b - no yr&month: ](#iter4b)**:[MLR all](#4bmlrall) || [ridge,lasso,enet](#4bregularization)

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import re

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Quick Cleaning 
(code and analysis taken from Part2_EDA)

In [5]:
resale = pd.read_csv('./datasets/resale_all.csv')
resale.drop([u'Unnamed: 0',u'geo_data',u'full_address','new full_address'],axis = 1,inplace=True)

#time
resale['date'] = pd.to_datetime(resale['month'])
resale['year'] = resale['date'].apply(lambda x:x.year)
resale['month'] = resale['date'].apply(lambda x:x.month)

#lease
resale['lease_commence_date'] = resale['lease_commence_date'].astype(int)
resale['calculated_remaining_lease'] = 99 - (resale['year'] - resale['lease_commence_date'])
resale['calculated_remaining_lease'] = [99 if yr >99 else yr for yr in resale['calculated_remaining_lease']]

#block 
resale['block_num'] = [re.findall('\d+',str(block_num))[0] for block_num in resale['block']]
resale['block_num'] = resale['block_num'].astype('int')

# flat model 
resale['flat_model'] = [model.upper() for model in resale['flat_model']]
resale['flat_model_PREMIUM'] = [1 if 'PREMIUM' in model else 0 for model in resale['flat_model']]
resale['flat_model_IMPROVED'] = [1 if 'IMPROVED' in model else 0 for model in resale['flat_model']]
resale['flat_model_MODEL_A'] = [1 if 'MODEL A' in model else 0 for model in resale['flat_model']]
resale.replace({'flat_model':{'MODEL A2':'OTHERS',
                              'PREMIUM APARTMENT':'APARTMENT',
                              'PREMIUM APARTMENT.':'APARTMENT',
                              'PREMIUM APARTMENT LOFT':'APARTMENT',
                              'MULTI GENERATION':'OTHERS',
                              '2-ROOM':'OTHERS',
                              'IMPROVED-MAISONETTE':'MAISONETTE',
                              'PREMIUM MAISONETTE':'MAISONETTE',
                              'IMPROVED-MAISONETTE':'MAISONETTE',
                              'MODEL A-MAISONETTE':'MAISONETTE',
                              'MODEL A':'OTHERS',
                              'IMPROVED':'OTHERS',
                              'TYPE S1': 'TYPE S',
                              'TYPE S2': 'TYPE S'
                              }},inplace=True)

#flat type
resale.replace({'flat_type':{'MULTI-GENERATION':'MULTI GENERATION'}},inplace=True)

#storey
resale['storey'] = [7 if floor == '06 TO 10' else 
                        13 if floor == '11 TO 15' else 
                        16 if floor == '19 TO 21' else
                        20 if floor[0] =='2' else 
                        30 if int(floor[0]) >= 3 else 
                        int(floor[:2]) for floor in resale['storey_range']]

resale.head()

C:\Users\ngyen\Anaconda2python2\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,...,formatted_address,postal_code,date,year,calculated_remaining_lease,block_num,flat_model_PREMIUM,flat_model_IMPROVED,flat_model_MODEL_A,storey
0,309,OTHERS,1 ROOM,31.0,1977,1,NaN,9000.0,10 TO 12,ANG MO KIO AVE 1,...,"309 Ang Mo Kio Ave 1, Singapore",NaN,1990-01-01,1990,86,309,0,1,0,10
1,309,OTHERS,1 ROOM,31.0,1977,1,NaN,6000.0,04 TO 06,ANG MO KIO AVE 1,...,"309 Ang Mo Kio Ave 1, Singapore",NaN,1990-01-01,1990,86,309,0,1,0,4
2,309,OTHERS,1 ROOM,31.0,1977,1,NaN,8000.0,10 TO 12,ANG MO KIO AVE 1,...,"309 Ang Mo Kio Ave 1, Singapore",NaN,1990-01-01,1990,86,309,0,1,0,10
3,309,OTHERS,1 ROOM,31.0,1977,1,NaN,6000.0,07 TO 09,ANG MO KIO AVE 1,...,"309 Ang Mo Kio Ave 1, Singapore",NaN,1990-01-01,1990,86,309,0,1,0,7
4,309,OTHERS,1 ROOM,31.0,1977,2,NaN,8000.0,04 TO 06,ANG MO KIO AVE 1,...,"309 Ang Mo Kio Ave 1, Singapore",NaN,1990-02-01,1990,86,309,0,1,0,4


In [2]:
# flatandmrt = pd.read_csv('./datasets/flatandmrt.csv')
# flatandmrt.drop([u'Unnamed: 0'],axis=1,inplace=True)
# flatandmrt['date'] = pd.to_datetime(flatandmrt['date'])
# flatandmrt['year'] = flatandmrt['date'].apply(lambda x:x.year)
# flatandmrt = flatandmrt[flatandmrt['year']>2004]
# flatandmrt.head()

KeyboardInterrupt: 

In [ ]:
flatandmrt = pd.read_csv('./datasets/flatandmrt_t0.csv')
flatandmrt.drop([u'Unnamed: 0'],axis=1,inplace=True)
flatandmrt['date'] = pd.to_datetime(flatandmrt['date'])
flatandmrt['year'] = flatandmrt['date'].apply(lambda x:x.year)
flatandmrt = flatandmrt[flatandmrt['year']>2004]
flatandmrt.head()

In [ ]:
flatandmrt.shape

In [ ]:
#Taking into consideration inflation across the years, we adjust the resale_price accordingly 
# inflation_rate is obtained from MAS website

inflation = pd.read_csv('./datasets/inflation_rate.csv')
inflation['value_of_1000_today'] = inflation['value_of_1000_today'].astype(int)/1000
inflate = inflation[['Year','value_of_1000_today']].values.tolist()

flatandmrt['adj_price'] = [price*change for p_year,price in zip(flatandmrt['year'],flatandmrt['resale_price']) 
                                        for [year,change] in inflate 
                                        if p_year == year]

In [ ]:
price_time = flatandmrt.groupby(['date']).mean()[['resale_price','adj_price']].reset_index()
price_time.columns = ['date','mean_resale_price','mean_adj_price']
print(price_time.shape)
price_time.head()

In [ ]:
flatandmrt = flatandmrt.merge(price_time,on='date')
flatandmrt['var_resale_price'] = flatandmrt['resale_price'] - flatandmrt['mean_resale_price']
flatandmrt['var_adj_price'] = flatandmrt['adj_price'] - flatandmrt['mean_adj_price']

In [ ]:
mrt= pd.read_csv('./datasets/mrt_all.csv')
mrt.drop(['Unnamed: 0'],axis=1,inplace=True)
mrt = mrt[mrt.Opening_date != 'TBA']
mrt.Opening_date = pd.to_datetime(mrt.Opening_date.apply(lambda x: '1-Jan-'+ x[2:] if len(x)==4 else x))
mrt.reset_index(drop=True,inplace=True)

In [ ]:
flatandmrt.head()

# Modelling 

### Standard Regression
**[Iteration 1: ](#iter1)** [SLR](#1slr)  ||  [MLR by category](#1mlrcat) <br> 
X = resale data <br>
y = resale['var_resale_price'] <br>

**[Iteration 2: ](#iter2)** [SLR](#2slr)  ||  [MLR by category](#2mlrcat) <br>
X = resale data <br>
y = resale['var_adj_price'] <br>


**[Iteration 3](#iter3)** <br>
X = resale data without time data<br>
y = resale['var_resale_price'] <br>
**[Iter3a - no yr:       ](#iter3a)**[MLR all](#3amlrall) || [ridge,lasso,enet](#3aregularization) <br>
**[Iter3b - no yr&month: ](#iter3b)**:[MLR all](#3bmlrall) || [ridge,lasso,enet](#3bregularization)

**[Iteration 4](#iter4)** <br>
X = resale data without time data<br>
y = resale['var_adj_price'] <br>
**[Iter4a - no yr:       ](#iter4a)**[MLR all](#4amlrall) || [ridge,lasso,enet](#4aregularization) <br>
**[Iter4b - no yr&month: ](#iter4b)**:[MLR all](#4bmlrall) || [ridge,lasso,enet](#4bregularization)

In [8]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, ElasticNet, ElasticNetCV, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [9]:
def Obtain_summary(estimator,X_train, y_train, X_test, y_test):
    scores = cross_val_score(estimator, X_train, y_train, cv=10)
    score_test = estimator.score(X_test,y_test)
    print('Mean of scores: {} | Std of scores: {}'.format(np.mean(scores), np.std(scores)))
    print('Test score: {}'.format(score_test))
    
    return {'mean_score':np.mean(scores), 
            'std_score':np.std(scores), 
            'scores':scores,
            'for_compare': [np.mean(scores),np.std(scores), score_test]
           }

def Obtain_coefficients(estimator,X_train,y_train,X):
    coefficient = pd.DataFrame({'variable':X.columns,
                                'coef':estimator.coef_,
                                'abs_coef':np.abs(estimator.coef_)})
    feat_coeff = coefficient[coefficient['abs_coef']>0]
    print('{} out of {} features are selected.' .format(feat_coeff.shape[0],len(X.columns)))
    return coefficient
    

def Add_model(model_name,summary,df):
    row = pd.DataFrame([model_name]+summary['for_compare'],
                       index=['model','train_mean_score','train_std_score','test_score']).T
    
    if type(df) == list:
        df = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])
        new_df = pd.concat([df,row])
    else:
        new_df = pd.concat([df,row])
        
    return new_df

<a id='iter1'></a>
## Iteration 1
X: is near mrt at that point in time <br>
y: var_resale_price or var_adj_price

In [17]:
flatnearmrt = flatandmrt.copy()

In [18]:
for i,station_name in list(zip(range(len(mrt)),flatnearmrt.columns[5:-6])):
    print(i,station_name)
    mrt_opening_date = mrt.loc[i,'Opening_date']
    have_mrt = flatnearmrt['date'].apply(lambda x: (x>mrt_opening_date))
    flatnearmrt[station_name]=flatnearmrt[station_name]*have_mrt

0 is_near_Choa Chu Kang MRT
1 is_near_Bukit Panjang MRT
2 is_near_South View LRT
3 is_near_Keat Hong LRT
4 is_near_Teck Whye LRT
5 is_near_Phoenix LRT
6 is_near_Petir LRT
7 is_near_Pending LRT
8 is_near_Bangkit LRT
9 is_near_Fajar LRT
10 is_near_Segar LRT
11 is_near_Jelapang LRT
12 is_near_Senja LRT
13 is_near_Ten Mile Junction LRT
14 is_near_Expo MRT (CGL)
15 is_near_Changi Airport MRT (CGL)
16 is_near_Bishan MRT (CCL)
17 is_near_Dhoby Ghaut MRT (CCL)
18 is_near_Paya Lebar MRT (CCL)
19 is_near_Buona Vista MRT (CCL)
20 is_near_HarbourFront MRT (CCL)
21 is_near_Serangoon MRT (CCL)
22 is_near_Bras Basah MRT (CCL)
23 is_near_Esplanade MRT (CCL)
24 is_near_Promenade MRT (CCL)
25 is_near_Nicoll Highway MRT (CCL)
26 is_near_Stadium MRT (CCL)
27 is_near_Mountbatten MRT (CCL)
28 is_near_Dakota MRT (CCL)
29 is_near_MacPherson MRT (CCL)
30 is_near_Tai Seng MRT (CCL)
31 is_near_Bartley MRT (CCL)
32 is_near_Lorong Chuan MRT (CCL)
33 is_near_Marymount MRT (CCL)
34 is_near_Caldecott MRT (CCL)
35 is_

In [21]:
[col for col in flatnearmrt.columns if 'price' in col]

['resale_price',
 'adj_price',
 'mean_resale_price',
 'mean_adj_price',
 'var_resale_price',
 'var_adj_price']

<a id='1slr'></a>
### Iteration 1a: Single Variable Linear Regression
want to see the effect of each single variable on resale_price

In [42]:
df = flatnearmrt[[col for col in flatnearmrt.columns if 'is_near' in col]+['year','var_resale_price','var_adj_price']]
train = df[(df['year']<2014)&(df['year']>2004)]
test = df[df['year']>=2014]
X = df.drop(['var_resale_price','var_adj_price','year'],axis=1)

In [24]:
summary_SLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for col in train.drop(['var_resale_price','var_adj_price','year'],axis=1).columns:
    X_train = train[[col]]
    y_train = train['var_resale_price']
    X_test = test[[col]]
    y_test = test['var_resale_price']
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    print(col)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_SLR = Add_model(col,summary_model,summary_SLR)
    print('--------------------------------------')

is_near_Choa Chu Kang MRT
Mean of scores: -0.00010643072296889766 | Std of scores: 7.436801141842945e-05
Test score: -6.341805701071124e-05
--------------------------------------
is_near_Bukit Panjang MRT
Mean of scores: -8.451234453672462e-05 | Std of scores: 9.205725983791998e-05
Test score: -3.711602449274487e-05
--------------------------------------
is_near_South View LRT
Mean of scores: -7.572434166112086e-05 | Std of scores: 8.696613852182724e-05
Test score: -6.449509550243704e-05
--------------------------------------
is_near_Keat Hong LRT
Mean of scores: 0.0005514169829759386 | Std of scores: 0.0004671338693140981
Test score: 0.0008075812118005787
--------------------------------------
is_near_Teck Whye LRT
Mean of scores: 0.0016572282790959015 | Std of scores: 0.0005608622339534432
Test score: 0.0015942718872358208
--------------------------------------
is_near_Phoenix LRT
Mean of scores: 0.00039396716121278266 | Std of scores: 0.00031324377857141827
Test score: 0.00069038546

Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Newton MRT (DTL)
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Tampines MRT (DTL)
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Bugis MRT (DTL)
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Expo MRT (DTL)
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Chinatown MRT (DTL)
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Little India MRT (DTL)
Mean of scores: -6.86655692521354e-05 | Std of scores: 7

Mean of scores: 0.00017031894144480252 | Std of scores: 0.0003208845998889996
Test score: 0.0014140664356323553
--------------------------------------
is_near_Bugis MRT (EWL)
Mean of scores: 0.00017338383270654179 | Std of scores: 0.0004991106182700597
Test score: 0.0002665932074701338
--------------------------------------
is_near_Tanjong Pagar MRT (EWL)
Mean of scores: 0.0022487158193820523 | Std of scores: 0.0014206397541500712
Test score: 0.015549797490254758
--------------------------------------
is_near_Outram Park MRT (EWL)
Mean of scores: 0.0018606369741057892 | Std of scores: 0.0016584762049555652
Test score: 0.013032420734536254
--------------------------------------
is_near_Tiong Bahru MRT (EWL)
Mean of scores: 0.008421391251604515 | Std of scores: 0.006303370506066234
Test score: 0.012384612812138984
--------------------------------------
is_near_Redhill MRT (EWL)
Mean of scores: 0.004766919640161171 | Std of scores: 0.002265360999602806
Test score: 0.00678377554506282
----

Mean of scores: 0.00014417728018014663 | Std of scores: 0.0006415898196576357
Test score: 0.0003789849600770445
--------------------------------------
is_near_Farrer Park MRT (NEL)
Mean of scores: 0.0016236683856941214 | Std of scores: 0.0017776643714086937
Test score: 0.005053533722673587
--------------------------------------
is_near_Boon Keng MRT (NEL)
Mean of scores: 0.0004917600785061315 | Std of scores: 0.0002643429154321702
Test score: -0.0014419040524642488
--------------------------------------
is_near_Potong Pasir MRT (NEL)
Mean of scores: 3.4488548898503614e-05 | Std of scores: 0.00015969206089623414
Test score: -3.563617623147941e-05
--------------------------------------
is_near_Woodleigh MRT (NEL)
Mean of scores: -9.118976739090901e-05 | Std of scores: 0.0001026742080489806
Test score: -6.765509070527642e-05
--------------------------------------
is_near_Serangoon MRT (NEL)
Mean of scores: 0.00012087516028760347 | Std of scores: 0.0006543185960681114
Test score: 0.0011507

Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Samudera LRT
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Nibong LRT
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Sumang LRT
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Soo Teck LRT
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Sengkang MRT
Mean of scores: 0.0053047891450225815 | Std of scores: 0.0014804349924380158
Test score: -0.0010278692128824485
--------------------------------------
is_near_Compassvale LRT
Mean of scores: 0.0013056032941945017 | Std of scores: 0.000373246278

In [25]:
summary_SLR.sort_values('train_mean_score',ascending=False).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Bishan MRT (NSL),0.017843,0.0013742,0.0218252
0,is_near_Yishun MRT (NSL),0.0177798,0.00254254,0.01436
0,is_near_Bishan MRT (CCL),0.00910868,0.00694268,0.0234198
0,is_near_Tiong Bahru MRT (EWL),0.00842139,0.00630337,0.0123846
0,is_near_Pasir Ris MRT (EWL),0.0076326,0.00140272,0.00253219
0,is_near_Sengkang MRT,0.00530479,0.00148043,-0.00102787
0,is_near_Sengkang MRT (NEL),0.00530479,0.00148043,-0.00102787
0,is_near_Lakeside MRT (EWL),0.00485515,0.00106528,0.00480038
0,is_near_Redhill MRT (EWL),0.00476692,0.00226536,0.00678378
0,is_near_Pending LRT,0.00354983,0.000849251,0.00308181


In [26]:
summary_SLR.sort_values('train_mean_score',ascending=True).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Admiralty MRT (NSL),-0.000293728,0.000800154,0.00048422
0,is_near_Woodlands MRT (NSL),-0.000141496,8.94438e-05,-2.48989e-05
0,is_near_Clementi MRT (EWL),-0.000116481,0.00011286,8.37494e-05
0,is_near_Choa Chu Kang MRT,-0.000106431,7.4368e-05,-6.34181e-05
0,is_near_Choa Chu Kang MRT (NSL),-0.000106431,7.4368e-05,-6.34181e-05
0,is_near_Woodleigh MRT (NEL),-9.11898e-05,0.000102674,-6.76551e-05
0,is_near_Bukit Panjang MRT,-8.45123e-05,9.20573e-05,-3.7116e-05
0,is_near_Eunos MRT (EWL),-8.10861e-05,7.83102e-05,8.01159e-08
0,is_near_Kembangan MRT (EWL),-7.93146e-05,9.48353e-05,1.22017e-05
0,is_near_Chinatown MRT (NEL),-7.717e-05,8.70697e-05,0.000862167


In [28]:
summary_SLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for col in train.drop(['var_resale_price','var_adj_price','year'],axis=1).columns:
    X_train = train[[col]]
    y_train = train['var_adj_price']
    X_test = test[[col]]
    y_test = test['var_adj_price']
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    print(col)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_SLR = Add_model(col,summary_model,summary_SLR)
    print('-----------------------------')

is_near_Choa Chu Kang MRT
Mean of scores: -0.00010734474319261667 | Std of scores: 8.637014377612217e-05
Test score: -0.0001354131733630748
-----------------------------
is_near_Bukit Panjang MRT
Mean of scores: -9.184062943822236e-05 | Std of scores: 8.582495180579569e-05
Test score: -3.675544172465628e-05
-----------------------------
is_near_South View LRT
Mean of scores: -7.88179495003205e-05 | Std of scores: 9.190520566429069e-05
Test score: -9.11799757539189e-05
-----------------------------
is_near_Keat Hong LRT
Mean of scores: 0.0006094451982151417 | Std of scores: 0.0004076944076208235
Test score: 0.0008868852592155418
-----------------------------
is_near_Teck Whye LRT
Mean of scores: 0.001757069100078179 | Std of scores: 0.0004988147296584674
Test score: 0.0016927077432458137
-----------------------------
is_near_Phoenix LRT
Mean of scores: 0.00042546399529932446 | Std of scores: 0.0003090507595435518
Test score: 0.0007681364348061104
-----------------------------
is_near_Pe

Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Tampines MRT (DTL)
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Bugis MRT (DTL)
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Expo MRT (DTL)
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Chinatown MRT (DTL)
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Little India MRT (DTL)
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_P

Mean of scores: 0.00011207086117644982 | Std of scores: 0.0005156205939870133
Test score: 0.001000136089175041
-----------------------------
is_near_Lavender MRT (EWL)
Mean of scores: 0.00017262447049199993 | Std of scores: 0.00031645271529891455
Test score: 0.0016339039138441034
-----------------------------
is_near_Bugis MRT (EWL)
Mean of scores: 0.0002047179399886634 | Std of scores: 0.0004324882463686607
Test score: 0.00028105764127350774
-----------------------------
is_near_Tanjong Pagar MRT (EWL)
Mean of scores: 0.002477342199759325 | Std of scores: 0.0012465123974221494
Test score: 0.01765473151341712
-----------------------------
is_near_Outram Park MRT (EWL)
Mean of scores: 0.0020721087376815773 | Std of scores: 0.0014183904428845804
Test score: 0.014801208001917754
-----------------------------
is_near_Tiong Bahru MRT (EWL)
Mean of scores: 0.00914040718432586 | Std of scores: 0.00519235828150387
Test score: 0.013299516882091968
-----------------------------
is_near_Redhill M

Mean of scores: -7.334575736034354e-05 | Std of scores: 8.777675041217131e-05
Test score: 0.001010335904464088
-----------------------------
is_near_Clarke Quay MRT (NEL)
Mean of scores: 3.5485263771173337e-05 | Std of scores: 0.00010342271353609371
Test score: 1.9565569093327184e-05
-----------------------------
is_near_Little India MRT (NEL)
Mean of scores: 0.00018797310849729686 | Std of scores: 0.0005252264675587588
Test score: 0.0004016833184488089
-----------------------------
is_near_Farrer Park MRT (NEL)
Mean of scores: 0.0017690349178489772 | Std of scores: 0.0015305054550935491
Test score: 0.005686286581910283
-----------------------------
is_near_Boon Keng MRT (NEL)
Mean of scores: 0.0005068040885065517 | Std of scores: 0.0003151554294907533
Test score: -0.0017933812579860044
-----------------------------
is_near_Potong Pasir MRT (NEL)
Mean of scores: 4.1333085634198685e-05 | Std of scores: 0.00015755948494709806
Test score: -5.240881942913944e-05
---------------------------

Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Punggol Point LRT
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Samudera LRT
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Nibong LRT
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Sumang LRT
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Soo Teck LRT
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Sengkang MRT
Mean of scores:

Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------
is_near_Bedok South MRT (TEL)
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
-----------------------------


In [29]:
summary_SLR.sort_values('train_mean_score',ascending=False).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Bishan MRT (NSL),0.0185516,0.00151501,0.0242235
0,is_near_Yishun MRT (NSL),0.0184963,0.00159059,0.0150552
0,is_near_Tiong Bahru MRT (EWL),0.00914041,0.00519236,0.0132995
0,is_near_Bishan MRT (CCL),0.0086683,0.00700755,0.024403
0,is_near_Pasir Ris MRT (EWL),0.00789193,0.00151212,0.00220017
0,is_near_Sengkang MRT,0.00551781,0.00137371,-0.00186908
0,is_near_Sengkang MRT (NEL),0.00551781,0.00137371,-0.00186908
0,is_near_Redhill MRT (EWL),0.0051935,0.00198228,0.00740423
0,is_near_Lakeside MRT (EWL),0.00507483,0.000976886,0.00510396
0,is_near_Pending LRT,0.00370948,0.000852001,0.00324851


In [30]:
summary_SLR.sort_values('train_mean_score',ascending=True).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Admiralty MRT (NSL),-0.00022733,0.000510695,0.0003832
0,is_near_Woodlands MRT (NSL),-0.000140055,0.000142356,-6.9547e-05
0,is_near_Choa Chu Kang MRT,-0.000107345,8.63701e-05,-0.000135413
0,is_near_Choa Chu Kang MRT (NSL),-0.000107345,8.63701e-05,-0.000135413
0,is_near_Clementi MRT (EWL),-0.000107227,7.87474e-05,3.53695e-05
0,is_near_Bukit Panjang MRT,-9.18406e-05,8.5825e-05,-3.67554e-05
0,is_near_Woodleigh MRT (NEL),-8.8433e-05,9.81353e-05,-6.16923e-05
0,is_near_Eunos MRT (EWL),-8.36717e-05,8.33159e-05,-3.81532e-05
0,is_near_Kembangan MRT (EWL),-8.08393e-05,0.000101226,1.61511e-05
0,is_near_South View LRT,-7.88179e-05,9.19052e-05,-9.118e-05


**Insights** <br>
From the summary scores, the factors that are quite important to var_price are: 
- if the mrt is near Bishan 
- near yishun (very negative score) (lol)

The factors that are not important are:
- strangely the places near town have a bad score. likely because of inbalance class

<a id='1mlrcat'></a>
### Iteration 1b: Multiple Variable Linear Regression by Category

In [34]:
set([col[-4:] for col in train.columns if 'is_near' in col])

{' LRT',
 ' MRT',
 'CCL)',
 'CGL)',
 'DTL)',
 'EWL)',
 'JRL)',
 'NEL)',
 'NSL)',
 'TEL)'}

In [37]:
summary_MLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for cat in ['LRT', 'CCL', 'CGL', 'DTL', 'EWL', 'JRL', 'NEL', 'NSL', 'TEL']:
    cat_pred = [col for col in train.columns if cat in col]
    X_train = train[cat_pred]
    y_train = train['var_resale_price']
    X_test = test[cat_pred]
    y_test = test['var_resale_price']

    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    print(cat)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_MLR = Add_model(cat,summary_model,summary_MLR)

LRT
Mean of scores: 0.020378127822342607 | Std of scores: 0.004959465015491482
Test score: -8.278900711989218e+20
CCL
Mean of scores: 0.0167381899162773 | Std of scores: 0.013070439361181498
Test score: 0.05851280387970659
CGL
Mean of scores: 2.7444793211939267e-05 | Std of scores: 0.00014953402439909542
Test score: 0.0007403073441917263
DTL
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
EWL
Mean of scores: 0.04391360238729062 | Std of scores: 0.012113601986250831
Test score: 0.05956263326913224
JRL
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
NEL
Mean of scores: 0.016780275396910026 | Std of scores: 0.0062939365280009285
Test score: 0.022945012539012044
NSL
Mean of scores: 0.048811243295995774 | Std of scores: 0.0028985756800527646
Test score: 0.045261314846811285
TEL
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0


In [38]:
summary_MLR.sort_values('test_score',ascending=False)

,model,train_mean_score,train_std_score,test_score
0,EWL,0.0439136,0.0121136,0.0595626
0,CCL,0.0167382,0.0130704,0.0585128
0,NSL,0.0488112,0.00289858,0.0452613
0,NEL,0.0167803,0.00629394,0.022945
0,CGL,2.74448e-05,0.000149534,0.000740307
0,DTL,-6.86656e-05,7.90061e-05,0
0,JRL,-6.86656e-05,7.90061e-05,0
0,TEL,-6.86656e-05,7.90061e-05,0
0,LRT,0.0203781,0.00495947,-8.2789e+20


In [39]:
summary_MLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for cat in ['LRT', 'CCL', 'CGL', 'DTL', 'EWL', 'JRL', 'NEL', 'NSL', 'TEL']:
    cat_pred = [col for col in train.columns if cat in col]
    X_train = train[cat_pred]
    y_train = train['var_adj_price']
    X_test = test[cat_pred]
    y_test = test['var_adj_price']

    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    print(cat)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_MLR = Add_model(cat,summary_model,summary_MLR)

LRT
Mean of scores: 0.02075320283985078 | Std of scores: 0.004403975276912133
Test score: -1.8040409995559871e+21
CCL
Mean of scores: 0.015357274178390801 | Std of scores: 0.012425958996697241
Test score: 0.05933390947105732
CGL
Mean of scores: 2.7243488599093534e-05 | Std of scores: 0.0001553146781945663
Test score: 0.0008714547676424989
DTL
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
EWL
Mean of scores: 0.0464552368172666 | Std of scores: 0.008783834075610168
Test score: 0.06408028814646982
JRL
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
NEL
Mean of scores: 0.01755636054257641 | Std of scores: 0.005345896572143991
Test score: 0.024202829648995342
NSL
Mean of scores: 0.05070496873285259 | Std of scores: 0.0030472800959037414
Test score: 0.048120950988408184
TEL
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score:

In [40]:
summary_MLR.sort_values('test_score',ascending=False)

,model,train_mean_score,train_std_score,test_score
0,EWL,0.0464552,0.00878383,0.0640803
0,CCL,0.0153573,0.012426,0.0593339
0,NSL,0.050705,0.00304728,0.048121
0,NEL,0.0175564,0.0053459,0.0242028
0,CGL,2.72435e-05,0.000155315,0.000871455
0,DTL,-7.06904e-05,7.95081e-05,1.11022e-16
0,JRL,-7.06904e-05,7.95081e-05,1.11022e-16
0,TEL,-7.06904e-05,7.95081e-05,1.11022e-16
0,LRT,0.0207532,0.00440398,-1.80404e+21


**Insights** <br>


<a id='1amlrall'></a>
### Iteration 1c: Multiple variable Linear regression: ALL 

In [45]:
cat_pred = [col for col in train.columns if 'is_near' in col]
X_train = train[cat_pred]
y_train = train['var_resale_price']
X_test = test[cat_pred]
y_test = test['var_resale_price']

In [46]:
ss=StandardScaler()
linreg = LinearRegression()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('linreg',linreg)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter1c_basic linreg_t=0',summary_model,[])
coeff = Obtain_coefficients(linreg,ss.fit_transform(X_train),y_train,X)

Mean of scores: 0.1264277192413771 | Std of scores: 0.02190865691666362
Test score: -2.613545843438955e+22
198 out of 245 features are selected.


In [48]:
coeff.sort_values('abs_coef',ascending=False)

,variable,coef,abs_coef
72,is_near_Jalan Besar MRT (DTL),-1.575437e+17,1.575437e+17
71,is_near_Bencoolen MRT (DTL),-1.515507e+17,1.515507e+17
77,is_near_Kaki Bukit MRT (DTL),-1.426411e+17,1.426411e+17
65,is_near_Tan Kah Kee MRT (DTL),1.378241e+17,1.378241e+17
40,is_near_Haw Par Villa MRT (CCL),-1.365714e+17,1.365714e+17
59,is_near_Bukit Panjang MRT (DTL),-1.315356e+17,1.315356e+17
60,is_near_Cashew MRT (DTL),-1.257157e+17,1.257157e+17
66,is_near_Stevens MRT (DTL),1.253239e+17,1.253239e+17
57,is_near_Botanic Gardens MRT (DTL),-1.084454e+17,1.084454e+17
63,is_near_King Albert Park MRT (DTL),-1.075539e+17,1.075539e+17


<a id='3aregularization'></a>
### Iteration 1d: Ridge, Lasso, enet 

In [49]:
ridge = Ridge()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('ridge',ridge)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3a_basic ridge_t=0',summary_model,summary)
coeff = Obtain_coefficients(ridge,ss.fit_transform(X_train),y_train,X)
coeff.sort_values('abs_coef',ascending=False)

Mean of scores: 0.12686126109069365 | Std of scores: 0.02156864175977908
Test score: 0.1212583850976634
121 out of 245 features are selected.


,variable,coef,abs_coef
174,is_near_Bishan MRT (NSL),13060.631673,13060.631673
104,is_near_Queenstown MRT (EWL),12753.641803,12753.641803
105,is_near_Commonwealth MRT (EWL),-11824.219739,11824.219739
170,is_near_Yishun MRT (NSL),-11058.777528,11058.777528
102,is_near_Tiong Bahru MRT (EWL),9416.013075,9416.013075
88,is_near_Pasir Ris MRT (EWL),9268.885650,9268.885650
176,is_near_Toa Payoh MRT (NSL),8431.610486,8431.610486
148,is_near_Farrer Park MRT (NEL),7225.836953,7225.836953
187,is_near_Cove LRT,6734.134656,6734.134656
155,is_near_Buangkok MRT (NEL),5616.938857,5616.938857


In [50]:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(ss.fit_transform(X_train),y_train)
lasso = Lasso(alpha=optimal_lasso.alpha_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('lasso',lasso)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3a_basic lasso_t=0',summary_model,summary)
coeff = Obtain_coefficients(lasso,ss.fit_transform(X_train),y_train,X)
coeff.sort_values('abs_coef',ascending=False)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Mean of scores: 0.12697565412491843 | Std of scores: 0.021189913287858922
Test score: 0.12115764583625555
114 out of 245 features are selected.


,variable,coef,abs_coef
174,is_near_Bishan MRT (NSL),12992.240959,12992.240959
104,is_near_Queenstown MRT (EWL),12561.639387,12561.639387
105,is_near_Commonwealth MRT (EWL),-11650.858900,11650.858900
170,is_near_Yishun MRT (NSL),-11033.089261,11033.089261
102,is_near_Tiong Bahru MRT (EWL),9375.993401,9375.993401
88,is_near_Pasir Ris MRT (EWL),9225.178635,9225.178635
176,is_near_Toa Payoh MRT (NSL),8304.237767,8304.237767
101,is_near_Outram Park MRT (EWL),7727.531332,7727.531332
148,is_near_Farrer Park MRT (NEL),7058.522764,7058.522764
187,is_near_Cove LRT,6587.844455,6587.844455


In [51]:
l1_ratios = np.linspace(0.01, 1.0, 25)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=30, cv=10,
                            verbose=1)
optimal_enet.fit(ss.fit_transform(X_train),y_train)
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('enet',enet)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3a_basic enet_t=0',summary_model,summary)
coeff = Obtain_coefficients(enet,ss.fit_transform(X_train),y_train,X)
coeff.sort_values('abs_coef',ascending=False)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Mean of scores: 0.12697788704919058 | Std of scores: 0.02118028410969505
Test score: 0.12115414798625401
114 out of 245 features are selected.


,variable,coef,abs_coef
174,is_near_Bishan MRT (NSL),12990.452983,12990.452983
104,is_near_Queenstown MRT (EWL),12556.678669,12556.678669
105,is_near_Commonwealth MRT (EWL),-11646.449699,11646.449699
170,is_near_Yishun MRT (NSL),-11032.488420,11032.488420
102,is_near_Tiong Bahru MRT (EWL),9374.930220,9374.930220
88,is_near_Pasir Ris MRT (EWL),9224.020398,9224.020398
176,is_near_Toa Payoh MRT (NSL),8300.943097,8300.943097
101,is_near_Outram Park MRT (EWL),7719.869863,7719.869863
148,is_near_Farrer Park MRT (NEL),7054.293863,7054.293863
187,is_near_Cove LRT,6583.992956,6583.992956


In [59]:
summary

,model,train_mean_score,train_std_score,test_score
0,iter1c_basic linreg_t=0,0.126428,0.0219087,-2.61355e+22
0,iter3a_basic ridge_t=0,0.126861,0.0215686,0.121258
0,iter3a_basic lasso_t=0,0.126976,0.0211899,0.121158
0,iter3a_basic enet_t=0,0.126978,0.0211803,0.121154


<a id='iter2'></a>
## Iteration 2
Looking at the potential of mrt 

In [10]:
def create_nearmrt_bytime(flatnearmrt,n_yr):
    year = timedelta(weeks=40, days=84, hours=23, minutes=50, seconds=600)
    for i,station_name in list(zip(range(len(mrt)),flatnearmrt.columns[5:-6])):
        print(i,station_name)
        mrt_opening_date = mrt.loc[i,'Opening_date']
        have_mrt = flatnearmrt['date'].apply(lambda x: (x>mrt_opening_date-(n_yr*year)))
        flatnearmrt[station_name+'_'+str(n_yr)]=flatnearmrt[station_name]*have_mrt
    return flatnearmrt

In [14]:
mrt.full_station_name

0           Choa Chu Kang MRT
1           Bukit Panjang MRT
2              South View LRT
3               Keat Hong LRT
4               Teck Whye LRT
5                 Phoenix LRT
6                   Petir LRT
7                 Pending LRT
8                 Bangkit LRT
9                   Fajar LRT
10                  Segar LRT
11               Jelapang LRT
12                  Senja LRT
13      Ten Mile Junction LRT
14                   Expo MRT
15         Changi Airport MRT
16                 Bishan MRT
17            Dhoby Ghaut MRT
18             Paya Lebar MRT
19            Buona Vista MRT
20           HarbourFront MRT
21              Serangoon MRT
22             Bras Basah MRT
23              Esplanade MRT
24              Promenade MRT
25         Nicoll Highway MRT
26                Stadium MRT
27            Mountbatten MRT
28                 Dakota MRT
29             MacPherson MRT
                ...          
215               Orchard MRT
216            Marina Bay MRT
217       

In [ ]:
flatnearmrt_t3 = create_nearmrt_bytime(flatandmrt,3)

0 is_near_Choa Chu Kang MRT
1 is_near_Bukit Panjang MRT
2 is_near_South View LRT
3 is_near_Keat Hong LRT
4 is_near_Teck Whye LRT
5 is_near_Phoenix LRT
6 is_near_Petir LRT
7 is_near_Pending LRT
8 is_near_Bangkit LRT
9 is_near_Fajar LRT
10 is_near_Segar LRT
11 is_near_Jelapang LRT
12 is_near_Senja LRT
13 is_near_Ten Mile Junction LRT
14 is_near_Expo MRT (CGL)
15 is_near_Changi Airport MRT (CGL)
16 is_near_Bishan MRT (CCL)
17 is_near_Dhoby Ghaut MRT (CCL)
18 is_near_Paya Lebar MRT (CCL)
19 is_near_Buona Vista MRT (CCL)
20 is_near_HarbourFront MRT (CCL)
21 is_near_Serangoon MRT (CCL)
22 is_near_Bras Basah MRT (CCL)
23 is_near_Esplanade MRT (CCL)
24 is_near_Promenade MRT (CCL)
25 is_near_Nicoll Highway MRT (CCL)
26 is_near_Stadium MRT (CCL)
27 is_near_Mountbatten MRT (CCL)
28 is_near_Dakota MRT (CCL)
29 is_near_MacPherson MRT (CCL)
30 is_near_Tai Seng MRT (CCL)
31 is_near_Bartley MRT (CCL)
32 is_near_Lorong Chuan MRT (CCL)
33 is_near_Marymount MRT (CCL)
34 is_near_Caldecott MRT (CCL)
35 is_

In [13]:
flatnearmrt_t3.head()

,street_name,latitude,longitude,date,resale_price,is_near_Choa Chu Kang MRT,is_near_Bukit Panjang MRT,is_near_South View LRT,is_near_Keat Hong LRT,is_near_Teck Whye LRT,...,is_near_Marina South MRT (TEL)_3,is_near_Gardens by the Bay MRT (TEL)_3,is_near_Tanjong Rhu MRT (TEL)_3,is_near_Katong Park MRT (TEL)_3,is_near_Amber MRT (TEL)_3,is_near_Marine Parade MRT (TEL)_3,is_near_Marine Terrace MRT (TEL)_3,is_near_Siglap MRT (TEL)_3,is_near_Bayshore MRT (TEL)_3,is_near_Bedok South MRT (TEL)_3
0,ANG MO KIO AVE 1,1.366208,103.841417,2005-02-01,200000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ANG MO KIO AVE 1,1.366208,103.841417,2005-02-01,170000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ANG MO KIO AVE 3,1.368210,103.837250,2005-02-01,230000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ANG MO KIO AVE 1,1.365628,103.840420,2005-02-01,200000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ANG MO KIO AVE 3,1.368866,103.837884,2005-02-01,170000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<a id='2slr'></a>
### Iteration 2a: Single Variable Linear Regression

In [30]:
[col for col in flatnearmrt_t3.columns if 'MRT' not in col and 'LRT' not in col]

['street_name',
 'latitude',
 'longitude',
 'date',
 'resale_price',
 'year',
 'adj_price',
 'mean_resale_price',
 'mean_adj_price',
 'var_resale_price',
 'var_adj_price']

In [15]:
df = flatnearmrt_t3[[col for col in flatnearmrt_t3.columns if '_3' in col]+['year','var_resale_price','var_adj_price']]
train = df[df['year']<2014]
test = df[df['year']>=2014]
X = df.drop(['var_resale_price','var_adj_price','year'],axis=1)

In [16]:
df.shape

(319975, 248)

In [37]:
summary_SLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for col in train.drop(['var_resale_price','var_adj_price','year'],axis=1).columns:
    X_train = train[[col]]
    y_train = train['var_resale_price']
    X_test = test[[col]]
    y_test = test['var_resale_price']
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    print(col)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_SLR = Add_model(col,summary_model,summary_SLR)
    print('--------------------------------------')

is_near_Choa Chu Kang MRT_3
Mean of scores: -0.00010643072296889766 | Std of scores: 7.436801141842945e-05
Test score: -6.341805701071124e-05
--------------------------------------
is_near_Bukit Panjang MRT_3
Mean of scores: -8.451234453672462e-05 | Std of scores: 9.205725983791998e-05
Test score: -3.711602449274487e-05
--------------------------------------
is_near_South View LRT_3
Mean of scores: -7.572434166112086e-05 | Std of scores: 8.696613852182724e-05
Test score: -6.449509550243704e-05
--------------------------------------
is_near_Keat Hong LRT_3
Mean of scores: 0.0005514169829759386 | Std of scores: 0.0004671338693140981
Test score: 0.0008075812118005787
--------------------------------------
is_near_Teck Whye LRT_3
Mean of scores: 0.0016572282790959015 | Std of scores: 0.0005608622339534432
Test score: 0.0015942718872358208
--------------------------------------
is_near_Phoenix LRT_3
Mean of scores: 0.00039396716121278266 | Std of scores: 0.00031324377857141827
Test score: 0

Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Bayfront MRT (CCL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Newton MRT (DTL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Tampines MRT (DTL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Bugis MRT (DTL)_3
Mean of scores: 4.728857218960769e-05 | Std of scores: 0.00019646859105798168
Test score: 0.000293267382034168
--------------------------------------
is_near_Expo MRT (DTL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Chinatown MRT (DTL)_3
Mean of scores: -8.7164416583

Mean of scores: 1.4323610877164933e-05 | Std of scores: 0.0002357824778529767
Test score: 0.0004915655771553507
--------------------------------------
is_near_Aljunied MRT (EWL)_3
Mean of scores: 3.885248054991131e-06 | Std of scores: 0.0002608873080608498
Test score: 0.00034328675602846026
--------------------------------------
is_near_Kallang MRT (EWL)_3
Mean of scores: 6.273998472499099e-05 | Std of scores: 0.0006617296205775854
Test score: 0.0009826055236853426
--------------------------------------
is_near_Lavender MRT (EWL)_3
Mean of scores: 0.00017031894144480252 | Std of scores: 0.0003208845998889996
Test score: 0.0014140664356323553
--------------------------------------
is_near_Bugis MRT (EWL)_3
Mean of scores: 0.00017338383270654179 | Std of scores: 0.0004991106182700597
Test score: 0.0002665932074701338
--------------------------------------
is_near_Tanjong Pagar MRT (EWL)_3
Mean of scores: 0.0022487158193820523 | Std of scores: 0.0014206397541500712
Test score: 0.015549797

Mean of scores: 0.0004128644694571415 | Std of scores: 0.00032151396970677554
Test score: 0.00022019674066697803
--------------------------------------
is_near_Chinatown MRT (NEL)_3
Mean of scores: -7.716998518714213e-05 | Std of scores: 8.70696679664296e-05
Test score: 0.0008621668940551608
--------------------------------------
is_near_Clarke Quay MRT (NEL)_3
Mean of scores: 4.019766360841004e-05 | Std of scores: 0.00011453194376524081
Test score: 2.7597913287547016e-05
--------------------------------------
is_near_Little India MRT (NEL)_3
Mean of scores: 0.00014417728018014663 | Std of scores: 0.0006415898196576357
Test score: 0.0003789849600770445
--------------------------------------
is_near_Farrer Park MRT (NEL)_3
Mean of scores: 0.0016236683856941214 | Std of scores: 0.0017776643714086937
Test score: 0.005053533722673587
--------------------------------------
is_near_Boon Keng MRT (NEL)_3
Mean of scores: 0.0004917600785061315 | Std of scores: 0.0002643429154321702
Test score: 

Mean of scores: 0.0002061809931543368 | Std of scores: 0.00014994354070158527
Test score: -0.0013651214502625741
--------------------------------------
is_near_Oasis LRT_3
Mean of scores: 0.0013457751676630215 | Std of scores: 0.0006799061033035123
Test score: -0.0022930955791911067
--------------------------------------
is_near_Damai LRT_3
Mean of scores: 0.0002453465312181491 | Std of scores: 0.00019581342299630183
Test score: -0.001619910090407295
--------------------------------------
is_near_Sam Kee LRT_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Punggol Point LRT_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Samudera LRT_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Nibong LRT_3
Mean of scores: -6.

Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Siglap MRT (TEL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Bayshore MRT (TEL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------
is_near_Bedok South MRT (TEL)_3
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
--------------------------------------


In [38]:
summary_SLR.sort_values('train_mean_score',ascending=False).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Bishan MRT (NSL)_3,0.017843,0.0013742,0.0218252
0,is_near_Yishun MRT (NSL)_3,0.0177798,0.00254254,0.01436
0,is_near_Bishan MRT (CCL)_3,0.0157914,0.00351463,0.0226141
0,is_near_Tiong Bahru MRT (EWL)_3,0.00842139,0.00630337,0.0123846
0,is_near_Pasir Ris MRT (EWL)_3,0.0076326,0.00140272,0.00253219
0,is_near_Sengkang MRT_3,0.00530479,0.00148043,-0.00102787
0,is_near_Sengkang MRT (NEL)_3,0.00530479,0.00148043,-0.00102787
0,is_near_Lakeside MRT (EWL)_3,0.00485515,0.00106528,0.00480038
0,is_near_Redhill MRT (EWL)_3,0.00476692,0.00226536,0.00678378
0,is_near_MacPherson MRT (CCL)_3,0.00360297,0.00177049,0.0019611


In [39]:
summary_SLR.sort_values('train_mean_score',ascending=True).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Admiralty MRT (NSL)_3,-0.000293728,0.000800154,0.00048422
0,is_near_Woodlands MRT (NSL)_3,-0.000141496,8.94438e-05,-2.48989e-05
0,is_near_Clementi MRT (EWL)_3,-0.000116481,0.00011286,8.37494e-05
0,is_near_Choa Chu Kang MRT_3,-0.000106431,7.4368e-05,-6.34181e-05
0,is_near_Choa Chu Kang MRT (NSL)_3,-0.000106431,7.4368e-05,-6.34181e-05
0,is_near_Woodleigh MRT (NEL)_3,-8.78024e-05,9.55317e-05,-5.92618e-05
0,is_near_Chinatown MRT (DTL)_3,-8.71644e-05,7.57808e-05,0.00075362
0,is_near_Bukit Panjang MRT_3,-8.45123e-05,9.20573e-05,-3.7116e-05
0,is_near_Eunos MRT (EWL)_3,-8.10861e-05,7.83102e-05,8.01159e-08
0,is_near_Kembangan MRT (EWL)_3,-7.93146e-05,9.48353e-05,1.22017e-05


In [40]:
summary_SLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for col in train.drop(['var_resale_price','var_adj_price','year'],axis=1).columns:
    X_train = train[[col]]
    y_train = train['var_adj_price']
    X_test = test[[col]]
    y_test = test['var_adj_price']
    
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    print(col)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_SLR = Add_model(col,summary_model,summary_SLR)
    print('--------------------------------------')

is_near_Choa Chu Kang MRT_3
Mean of scores: -0.00010734474319261667 | Std of scores: 8.637014377612217e-05
Test score: -0.0001354131733630748
--------------------------------------
is_near_Bukit Panjang MRT_3
Mean of scores: -9.184062943822236e-05 | Std of scores: 8.582495180579569e-05
Test score: -3.675544172465628e-05
--------------------------------------
is_near_South View LRT_3
Mean of scores: -7.88179495003205e-05 | Std of scores: 9.190520566429069e-05
Test score: -9.11799757539189e-05
--------------------------------------
is_near_Keat Hong LRT_3
Mean of scores: 0.0006094451982151417 | Std of scores: 0.0004076944076208235
Test score: 0.0008868852592155418
--------------------------------------
is_near_Teck Whye LRT_3
Mean of scores: 0.001757069100078179 | Std of scores: 0.0004988147296584674
Test score: 0.0016927077432458137
--------------------------------------
is_near_Phoenix LRT_3
Mean of scores: 0.00042546399529932446 | Std of scores: 0.0003090507595435518
Test score: 0.000

Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Marina Bay MRT (CCL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Bayfront MRT (CCL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Newton MRT (DTL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Tampines MRT (DTL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Bugis MRT (DTL)_3
Mean of scores: 3.851719981853297e-05 | Std of scores: 0.00017715129554561707
Test score: 0.000292

Mean of scores: 0.0004565958196166009 | Std of scores: 0.000503455571509129
Test score: 0.00027870951017106016
--------------------------------------
is_near_Bedok MRT (EWL)_3
Mean of scores: 0.0025535300856163847 | Std of scores: 0.0008433491521769515
Test score: 0.0011943233657560892
--------------------------------------
is_near_Kembangan MRT (EWL)_3
Mean of scores: -8.083926544492082e-05 | Std of scores: 0.00010122635278550834
Test score: 1.6151117021023786e-05
--------------------------------------
is_near_Eunos MRT (EWL)_3
Mean of scores: -8.367166531598746e-05 | Std of scores: 8.331591865504823e-05
Test score: -3.815318978328541e-05
--------------------------------------
is_near_Paya Lebar MRT (EWL)_3
Mean of scores: -3.079541448736123e-06 | Std of scores: 0.0002032970100408953
Test score: 0.0005533122279610669
--------------------------------------
is_near_Aljunied MRT (EWL)_3
Mean of scores: 4.004308322724803e-06 | Std of scores: 0.0002324080932460598
Test score: 0.00036975303

Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Jurong Pier MRT (JRL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Nanyang Gateway MRT (JRL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Nanyang Crescent MRT (JRL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Peng Kang Hill MRT (JRL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Dhoby Ghaut MRT (NEL)_3
Mean of scores: -7.051660849725084e-05 | Std of scores: 9.0640522702

Mean of scores: -7.289442254534428e-05 | Std of scores: 7.790743073837827e-05
Test score: -4.961918603996196e-06
--------------------------------------
is_near_Marina Bay MRT (NSL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Marina South Pier MRT (NSL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Punggol MRT_3
Mean of scores: 0.0016215962487782342 | Std of scores: 0.0006926116581406459
Test score: -0.0009324065762794653
--------------------------------------
is_near_Cove LRT_3
Mean of scores: 0.003305224861047773 | Std of scores: 0.0011438134199061084
Test score: -0.002686097818240585
--------------------------------------
is_near_Meridian LRT_3
Mean of scores: 0.002916474057653884 | Std of scores: 0.0010376478201427743
Test score: -0.003153197055817847
-

Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Great World MRT (TEL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Havelock MRT (TEL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Maxwell MRT (TEL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Shenton Way MRT (TEL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
--------------------------------------
is_near_Marina South MRT (TEL)_3
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test sco

In [41]:
summary_SLR.sort_values('test_score',ascending=False).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Bishan MRT (CCL)_3,0.0158888,0.00351374,0.0246026
0,is_near_Bishan MRT (NSL)_3,0.0185516,0.00151501,0.0242235
0,is_near_Tanjong Pagar MRT (EWL)_3,0.00247734,0.00124651,0.0176547
0,is_near_Yishun MRT (NSL)_3,0.0184963,0.00159059,0.0150552
0,is_near_Outram Park MRT (EWL)_3,0.00207211,0.00141839,0.0148012
0,is_near_Outram Park MRT (NEL)_3,0.00207211,0.00141839,0.0148012
0,is_near_Tiong Bahru MRT (EWL)_3,0.00914041,0.00519236,0.0132995
0,is_near_Beauty World MRT (DTL)_3,0.00042744,0.00097511,0.00840453
0,is_near_Redhill MRT (EWL)_3,0.0051935,0.00198228,0.00740423
0,is_near_Buona Vista MRT (CCL)_3,0.00104991,0.00119414,0.00718055


In [42]:
summary_SLR.sort_values('test_score',ascending=True).head(20)

,model,train_mean_score,train_std_score,test_score
0,is_near_Meridian LRT_3,0.00291647,0.00103765,-0.0031532
0,is_near_Oasis LRT_3,0.0012845,0.000679579,-0.00278502
0,is_near_Fernvale LRT_3,0.000446552,0.00026718,-0.00277607
0,is_near_Cove LRT_3,0.00330522,0.00114381,-0.0026861
0,is_near_Yew Tee MRT (NSL)_3,0.00180305,0.00123922,-0.00259456
0,is_near_Commonwealth MRT (EWL)_3,0.000891685,0.00123643,-0.00245443
0,is_near_Sembawang MRT (NSL)_3,0.00064568,0.000691005,-0.00204451
0,is_near_Coral Edge LRT_3,0.00150056,0.000583334,-0.00203609
0,is_near_Damai LRT_3,0.000222741,0.000182739,-0.00190016
0,is_near_Sengkang MRT (NEL)_3,0.00551781,0.00137371,-0.00186908


**Insights** <br>
From the summary scores, the factors that are quite important to adj_price are:
- floor_area_sqm
- 3 room, 5 room, executive etc --> flat_type
- lease_commence_date
- new generation, apartments, premium etc --> flat_model (a few factors only)
- Certain years
- Calculated_remaining_lease
- Storey height

The factors that are not important are:
- month
- certain towns
- certain years

Effect on adj_price is higher than effect on resale_price

<a id='2mlrcat'></a>
### Iteration 2b: Multiple Variable Linear Regression by Category

In [43]:
summary_MLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for cat in ['LRT', 'CCL', 'CGL', 'DTL', 'EWL', 'JRL', 'NEL', 'NSL', 'TEL']:
    cat_pred = [col for col in train.columns if cat in col]
    X_train = train[cat_pred]
    y_train = train['var_resale_price']
    X_test = test[cat_pred]
    y_test = test['var_resale_price']

    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    print(cat)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_MLR = Add_model(cat,summary_model,summary_MLR)

LRT
Mean of scores: 0.02079614507628095 | Std of scores: 0.005136702385212433
Test score: -4.312547128873783e+22
CCL
Mean of scores: 0.03135462592327731 | Std of scores: 0.01219156539060612
Test score: 0.056681081316695026
CGL
Mean of scores: 2.7444793211939267e-05 | Std of scores: 0.00014953402439909542
Test score: 0.0007403073441917263
DTL
Mean of scores: 0.0008445864279495119 | Std of scores: 0.0014938944841184135
Test score: -7.397360682166426e+23
EWL
Mean of scores: 0.043755528965410836 | Std of scores: 0.012113306253668072
Test score: 0.059771009324202866
JRL
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0
NEL
Mean of scores: 0.01692827854495661 | Std of scores: 0.0059951213894633
Test score: 0.023140448014678072
NSL
Mean of scores: 0.048811243295995774 | Std of scores: 0.0028985756800527646
Test score: -25911051093.04326
TEL
Mean of scores: -6.86655692521354e-05 | Std of scores: 7.90060741551768e-05
Test score: 0.0


In [44]:
summary_MLR.sort_values('test_score',ascending = False)

,model,train_mean_score,train_std_score,test_score
0,EWL,0.0437555,0.0121133,0.059771
0,CCL,0.0313546,0.0121916,0.0566811
0,NEL,0.0169283,0.00599512,0.0231404
0,CGL,2.74448e-05,0.000149534,0.000740307
0,JRL,-6.86656e-05,7.90061e-05,0
0,TEL,-6.86656e-05,7.90061e-05,0
0,NSL,0.0488112,0.00289858,-2.59111e+10
0,LRT,0.0207961,0.0051367,-4.31255e+22
0,DTL,0.000844586,0.00149389,-7.39736e+23


In [45]:
summary_MLR = pd.DataFrame(columns=['model','train_mean_score','train_std_score','test_score'])

for cat in ['LRT', 'CCL', 'CGL', 'DTL', 'EWL', 'JRL', 'NEL', 'NSL', 'TEL']:
    cat_pred = [col for col in train.columns if cat in col]
    X_train = train[cat_pred]
    y_train = train['var_adj_price']
    X_test = test[cat_pred]
    y_test = test['var_adj_price']

    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    
    print(cat)
    summary_model = Obtain_summary(linreg,X_train, y_train, X_test, y_test)
    summary_MLR = Add_model(cat,summary_model,summary_MLR)

LRT
Mean of scores: 0.021171233343867845 | Std of scores: 0.00456895325542766
Test score: -8.87132364211306e+22
CCL
Mean of scores: 0.030874600344306335 | Std of scores: 0.011831764037388387
Test score: 0.060420173519893594
CGL
Mean of scores: 2.7243488599093534e-05 | Std of scores: 0.0001553146781945663
Test score: 0.0008714547676424989
DTL
Mean of scores: 0.0006421040951738565 | Std of scores: 0.001120816793142892
Test score: -1.5020704702365876e+24
EWL
Mean of scores: 0.046359117435305075 | Std of scores: 0.00874767061614387
Test score: 0.06430798798337034
JRL
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.1102230246251565e-16
NEL
Mean of scores: 0.017773782550303707 | Std of scores: 0.004989806588717497
Test score: 0.02440396008157997
NSL
Mean of scores: 0.05070496873285259 | Std of scores: 0.0030472800959037414
Test score: -58089952224.63651
TEL
Mean of scores: -7.069043732002011e-05 | Std of scores: 7.950813007334506e-05
Test score: 1.

In [46]:
summary_MLR.sort_values('test_score',ascending = False)

,model,train_mean_score,train_std_score,test_score
0,EWL,0.0463591,0.00874767,0.064308
0,CCL,0.0308746,0.0118318,0.0604202
0,NEL,0.0177738,0.00498981,0.024404
0,CGL,2.72435e-05,0.000155315,0.000871455
0,JRL,-7.06904e-05,7.95081e-05,1.11022e-16
0,TEL,-7.06904e-05,7.95081e-05,1.11022e-16
0,NSL,0.050705,0.00304728,-5.809e+10
0,LRT,0.0211712,0.00456895,-8.87132e+22
0,DTL,0.000642104,0.00112082,-1.50207e+24


### Iteration 2c: Multiple variable Linear regression: ALL 

In [17]:
cat_pred = [col for col in train.columns if 'is_near' in col]
X_train = train[cat_pred]
y_train = train['var_resale_price']
X_test = test[cat_pred]
y_test = test['var_resale_price']

In [19]:
ss=StandardScaler()
linreg = LinearRegression()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('linreg',linreg)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter2c_basic linreg_t=3',summary_model,[])
coeff = Obtain_coefficients(linreg,ss.fit_transform(X_train),y_train,X)

Mean of scores: 0.13214963189563703 | Std of scores: 0.022809128783340474
Test score: -8.793750957972273e+21
197 out of 245 features are selected.


### Iteration 2d: Ridge, Lasso, enet 

In [20]:
ridge = Ridge()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('ridge',ridge)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter2d_basic ridge_t=3',summary_model,summary)
coeff = Obtain_coefficients(ridge,ss.fit_transform(X_train),y_train,X)

Mean of scores: 0.1323826173335092 | Std of scores: 0.022739493754114357
Test score: 0.12985370209979907
134 out of 245 features are selected.


NameError: name 'Analyse_coef' is not defined

In [21]:
coeff.sort_values('abs_coef',ascending=False)

,variable,coef,abs_coef
104,is_near_Queenstown MRT (EWL)_3,12102.874278,12102.874278
170,is_near_Yishun MRT (NSL)_3,-10999.708851,10999.708851
174,is_near_Bishan MRT (NSL)_3,10997.096933,10997.096933
105,is_near_Commonwealth MRT (EWL)_3,-10951.299143,10951.299143
102,is_near_Tiong Bahru MRT (EWL)_3,9436.347166,9436.347166
88,is_near_Pasir Ris MRT (EWL)_3,9306.513265,9306.513265
176,is_near_Toa Payoh MRT (NSL)_3,8251.678708,8251.678708
155,is_near_Buangkok MRT (NEL)_3,6767.593638,6767.593638
187,is_near_Cove LRT_3,6608.984097,6608.984097
148,is_near_Farrer Park MRT (NEL)_3,6563.383992,6563.383992


In [22]:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(ss.fit_transform(X_train),y_train)
lasso = Lasso(alpha=optimal_lasso.alpha_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('lasso',lasso)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter2d_basic lasso_t=3',summary_model,summary)
coeff = Obtain_coefficients(lasso,ss.fit_transform(X_train),y_train,X)
coeff.sort_values('abs_coef',ascending=False)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Mean of scores: 0.13244315301389004 | Std of scores: 0.022574927465933033
Test score: 0.12990215530002824
131 out of 245 features are selected.


,variable,coef,abs_coef
104,is_near_Queenstown MRT (EWL)_3,12014.182346,12014.182346
170,is_near_Yishun MRT (NSL)_3,-10988.444086,10988.444086
174,is_near_Bishan MRT (NSL)_3,10966.082165,10966.082165
105,is_near_Commonwealth MRT (EWL)_3,-10871.856812,10871.856812
102,is_near_Tiong Bahru MRT (EWL)_3,9417.617447,9417.617447
88,is_near_Pasir Ris MRT (EWL)_3,9286.057637,9286.057637
176,is_near_Toa Payoh MRT (NSL)_3,8195.720580,8195.720580
101,is_near_Outram Park MRT (EWL)_3,7994.359770,7994.359770
155,is_near_Buangkok MRT (NEL)_3,6708.800905,6708.800905
187,is_near_Cove LRT_3,6535.919874,6535.919874


In [23]:
l1_ratios = np.linspace(0.01, 1.0, 25)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=30, cv=10,
                            verbose=1)
optimal_enet.fit(ss.fit_transform(X_train),y_train)
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('enet',enet)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter2d_basic enet_t=3',summary_model,summary)
coeff = Obtain_coefficients(enet,ss.fit_transform(X_train),y_train,X)
coeff.sort_values('abs_coef',ascending=False)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Mean of scores: 0.1324468434362056 | Std of scores: 0.022561077312388867
Test score: 0.12990327733620022
131 out of 245 features are selected.


,variable,coef,abs_coef
104,is_near_Queenstown MRT (EWL)_3,12006.171231,12006.171231
170,is_near_Yishun MRT (NSL)_3,-10987.471821,10987.471821
174,is_near_Bishan MRT (NSL)_3,10963.250511,10963.250511
105,is_near_Commonwealth MRT (EWL)_3,-10864.727927,10864.727927
102,is_near_Tiong Bahru MRT (EWL)_3,9415.911996,9415.911996
88,is_near_Pasir Ris MRT (EWL)_3,9284.191731,9284.191731
176,is_near_Toa Payoh MRT (NSL)_3,8190.668607,8190.668607
101,is_near_Outram Park MRT (EWL)_3,7979.655386,7979.655386
155,is_near_Buangkok MRT (NEL)_3,6703.504700,6703.504700
187,is_near_Cove LRT_3,6529.336483,6529.336483


In [24]:
summary

,model,train_mean_score,train_std_score,test_score
0,iter2c_basic linreg_t=3,0.13215,0.0228091,-8.79375e+21
0,iter2d_basic ridge_t=3,0.132383,0.0227395,0.129854
0,iter2d_basic lasso_t=3,0.132443,0.0225749,0.129902
0,iter2d_basic enet_t=3,0.132447,0.0225611,0.129903


<a id='iter3b'></a>
## Iteration 3

In [25]:
df = flatnearmrt_t3[[col for col in flatnearmrt_t3.columns if 'is_near' in col]+['year','var_resale_price','var_adj_price']]
train = df[df['year']<2014]
test = df[df['year']>=2014]
X = df.drop(['var_resale_price','var_adj_price','year'],axis=1)

In [26]:
cat_pred = [col for col in train.columns if 'is_near' in col]
X_train = train[cat_pred]
y_train = train['var_resale_price']
X_test = test[cat_pred]
y_test = test['var_resale_price']

In [27]:
ridge = Ridge()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('ridge',ridge)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter2d_basic ridge_t=3',summary_model,summary)
coeff = Obtain_coefficients(ridge,ss.fit_transform(X_train),y_train,X)

MemoryError: 

In [ ]:
coeff.sort_values('abs_coef',ascending=False)

In [ ]:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(ss.fit_transform(X_train),y_train)
lasso = Lasso(alpha=optimal_lasso.alpha_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('lasso',lasso)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3b_basic lasso',summary_model,summary)
coeff = Obtain_coefficients(lasso,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 25)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=30, cv=10,
                            verbose=1)
optimal_enet.fit(ss.fit_transform(X_train),y_train)
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('enet',enet)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3b_basic enet',summary_model,summary)
coeff = Obtain_coefficients(enet,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

##### Insights 

In [ ]:
summary.sort_values('train_mean_score',ascending = False)

<a id='iter4a'></a>
### Iteration 4a

In [ ]:
df = pd.concat([df1,df2,df3[[col for col in df3.columns if 'year' not in col]],df4,df5,resale[['var_resale_price','var_adj_price','year']]],axis=1)
train = df[(df['year']<2014)&(df['year']>2004)]
test = df[df['year']>=2014]
X = df.drop(['var_resale_price','var_adj_price','year'],axis=1)
X_train = train.drop(['var_resale_price','var_adj_price','year'],axis=1)
y_train = train['var_adj_price']
X_test = test.drop(['var_resale_price','var_adj_price','year'],axis=1)
y_test = test['var_adj_price']


<a id='4amlrall'></a>
#### Iteration 4a: Multiple variable Linear regression: ALL 

In [ ]:
linreg = LinearRegression()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('linreg',linreg)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter4a_basic linreg',summary_model,[])
coeff = Obtain_coefficients(linreg,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

<a id='4aregularization'></a>
#### Iteration 4a: Ridge, Lasso, enet 

In [ ]:
ridge = Ridge()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('ridge',ridge)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter4a_basic ridge',summary_model,summary)
coeff = Obtain_coefficients(ridge,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

In [ ]:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(ss.fit_transform(X_train),y_train)
lasso = Lasso(alpha=optimal_lasso.alpha_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('lasso',lasso)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter4a_basic lasso',summary_model,summary)
coeff = Obtain_coefficients(lasso,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 25)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=30, cv=10,
                            verbose=1)
optimal_enet.fit(ss.fit_transform(X_train),y_train)
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('enet',enet)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter4a_basic enet',summary_model,summary)
coeff = Obtain_coefficients(enet,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

<a id='iter4b'></a>
### Iteration 4b

In [ ]:
df = pd.concat([df1,df2,df3[[col for col in df3.columns if 'year' not in col and 'month' not in col]],df4,df5,resale[['var_resale_price','var_adj_price','year']]],axis=1)

train = df[(df['year']<2014)&(df['year']>2004)]
test = df[df['year']>=2014]

X = df.drop(['var_resale_price','var_adj_price','year'],axis=1)
X_train = train.drop(['var_resale_price','var_adj_price','year'],axis=1)
y_train = train['var_adj_price']
X_test = test.drop(['var_resale_price','var_adj_price','year'],axis=1)
y_test = test['var_adj_price']


<a id='4bmlrall'></a>
#### Iteration 4b: Multiple variable Linear regression: ALL 

In [ ]:
linreg = LinearRegression()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('linreg',linreg)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter4b_basic linreg',summary_model,summary)
coeff = Obtain_coefficients(linreg,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

<a id='4bregularization'></a>
#### Iteration 4b: Ridge, Lasso, enet

In [ ]:
ridge = Ridge()
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('ridge',ridge)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter4b_basic ridge',summary_model,summary)
coeff = Obtain_coefficients(ridge,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

In [ ]:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(ss.fit_transform(X_train),y_train)
lasso = Lasso(alpha=optimal_lasso.alpha_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('lasso',lasso)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3b_basic lasso',summary_model,summary)
coeff = Obtain_coefficients(lasso,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 25)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=30, cv=10,
                            verbose=1)
optimal_enet.fit(ss.fit_transform(X_train),y_train)
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('enet',enet)])
pipe.fit(X_train,y_train)

summary_model = Obtain_summary(pipe,X_train, y_train, X_test, y_test)
summary = Add_model('iter3b_basic enet',summary_model,summary)
coeff = Obtain_coefficients(enet,ss.fit_transform(X_train),y_train,X)
Analyse_coef(coeff,X)

## Insights

In [ ]:
summary.sort_values('test_score',ascending = False)

Iter3a: (all X variables except year variables)           + (y = var resale price)<br>
Iter3b: (all X variables except year and month variables) + (y = var resale price)<br>
Iter4a: (all X variables except year variables)           + (y = var adj price)<br>
Iter4b: (all X variables except year and month variables) + (y = var adj price)<br>

From the `summary` above, we have the following insights: 
- Immediately the score makes sense as time is not taken into consideration but the score is equally good using a time series train-test-split method 
-



